In [1]:
import requests
def download_data()->dict: #傳出dict
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    response = requests.get(url) #response是實體
    if response.status_code == 200:
        print('下載成功')
    return response.json()

In [8]:
def jsonDict_csvList(json)->list[dict]:
    '''
    - 由online json viewer看出locaion下的21個地點資料都是dict
    - 傳入json的資料結構，並取出需要的資料
    - 把json資料結構轉成python list，而內部是dict結構
    '''
    location = json['cwbopendata']['dataset']['location'] #location是一個list資料結構，是我們要的，因此用變數location來接收
    weather_list = [] #希望建立外部為list，而內部是dict的資料結構
    for item in location: #既然location是list也就是sequence，因此可以用for in來抓資料
        city_item = {} #希望建立內部是dict的資料結構
        city_item['城市'] = item['locationName']
        city_item['起始時間'] = item['weatherElement'][1]['time'][0]['startTime'] #但由online json viewer看出weatherElement是一個索引編號，是一個list結構
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        weather_list.append(city_item)
    return weather_list


In [ ]:
json_data = download_data()
csv_list = jsonDict_csvList(json_data)
csv_list